In [0]:
%sql
CREATE TABLE deviceData (
  _id STRING,
  createTimestamp BIGINT,
  type STRING,
  attribute STRING,
  relatedGroupId STRING,
  tagIds ARRAY<STRING>,
  communicationStatus STRING,
  customerId STRING,
  dealerId STRING,
  L1DealerId STRING,
  L2DealerId STRING,
  L3DealerId STRING,
  divisionId STRING,
  onContract BOOLEAN,
  purchaseDate BIGINT,
  equipmentId STRING,
  isSNAP BOOLEAN,
  servicingDealerId STRING,
  SAP_name STRING,
  SAP_location STRING,

  deviceGeneral STRUCT<
    relatedAgentId: STRING,
    modelName: STRING,
    serialNumber: STRING,
    familyName: STRING,
    friendlyName: STRING,
    dispErrorCode: INT
  >,

  printerDevice STRUCT<
    connectionSetting: STRUCT<
      ipAddress: STRUCT<
        value: STRING,
        ipToLong: BIGINT
      >,
      port: STRUCT<
        http: INT,
        https: INT
      >,
      snmpSetting: STRUCT<
        snmpVersion: STRING,
        retry: INT,
        timeout: INT,
        getCommunityString: STRING,
        contextName: STRING,
        userName: STRING,
        securityAuthType: STRING,
        securityAuthKey: STRING,
        securityPrivType: STRING,
        securityPrivKey: STRING
      >
    >,
    macAddress: STRING,
    statusCode: INT,
    status: ARRAY<STRING>,
    troubleCode: INT,
    dskFlag: BOOLEAN,
    location: STRING,
    firmwareVersion: STRING,
    subnetMask: STRING,
    gateway: STRING,
    lastStatusUpdate: BIGINT,
    lastSuppliesUpdate: BIGINT,
    lastCounterUpdate: BIGINT,
    lastBasicUpdate: BIGINT,
    lifeCount: BIGINT,
    detectedDeviceType: STRING,
    productFamilyName: STRING,

    forecastingList: ARRAY<STRUCT<
      color: STRING,
      printCount: INT,
      tonerEndDate: TIMESTAMP,
      tonerNumber: INT,
      tonerRemaining: INT
    >>,

    interface: STRUCT<
      ethernetList: ARRAY<STRUCT<
        id: STRING,
        type: STRING,
        address: STRING
      >>,
      ipList: ARRAY<STRUCT<
        address: STRING,
        ethernetId: STRING,
        subnetMask: STRING,
        defaultRoute: STRING
      >>
    >,

    maintenanceCodeList: ARRAY<STRUCT<
      id: STRING,
      value: STRING
    >>,

    serviceCodeList: ARRAY<STRUCT<
      id: STRING,
      value: STRING
    >>,

    inTrayList: ARRAY<STRUCT<
      name: STRING,
      id: STRING,
      description: STRING,
      mediaType: STRING,
      modelName: STRING,
      amount: STRUCT<
        state: STRING,
        unit: STRING,
        capacity: STRING,
        typical: STRING
      >,
      mediaSizeHeight: STRING,
      mediaSize: STRING,
      mediaSizeWidth: STRING,
      mediaSizeUnit: STRING,
      virtual: STRING,
      inserter: STRING,
      mediaSizeName: STRING,
      manual: STRING,
      mediaName: STRING,
      mediaDimFeed: STRING,
      mediaDimXFeed: STRING
    >>,

    outTrayList: ARRAY<STRUCT<
      name: STRING,
      id: STRING,
      description: STRING,
      deliveryOrientation: STRING,
      modelName: STRING,
      amount: STRUCT<
        state: STRING,
        unit: STRING,
        capacity: STRING,
        typical: STRING
      >
    >>,

    disposalMarkerList: ARRAY<STRUCT<
      type: STRING,
      description: STRING,
      color: STRING,
      amount: STRUCT<
        state: STRING,
        unit: STRING,
        capacity: STRING,
        typical: STRING
      >
    >>,

    markerList: ARRAY<STRUCT<
      type: STRING,
      description: STRING,
      color: STRING,
      amount: STRUCT<
        state: STRING,
        unit: STRING,
        capacity: STRING,
        typical: STRING
      >
    >>
  >
)
USING DELTA;


In [0]:
describe deviceData

In [0]:
INSERT INTO deviceData
SELECT
  concat('dev_', id) AS _id,
  unix_timestamp() * 1000 AS createTimestamp,
  'PRINTER' AS type,
  'ACTIVE' AS attribute,
  concat('grp_', id % 10) AS relatedGroupId,
  array(concat('tag_', id % 5)) AS tagIds,
  CASE WHEN id % 2 = 0 THEN 'ONLINE' ELSE 'OFFLINE' END AS communicationStatus,
  concat('cust_', id % 20) AS customerId,
  concat('dealer_', id % 5) AS dealerId,
  concat('L1_', id % 3) AS L1DealerId,
  concat('L2_', id % 3) AS L2DealerId,
  concat('L3_', id % 3) AS L3DealerId,
  concat('div_', id % 4) AS divisionId,
  id % 2 = 0 AS onContract,
  unix_timestamp() * 1000 AS purchaseDate,
  concat('equip_', id) AS equipmentId,
  id % 3 = 0 AS isSNAP,
  concat('svc_', id % 5) AS servicingDealerId,
  'SAP_DEVICE' AS SAP_name,
  'BLR' AS SAP_location,

  named_struct(
    'relatedAgentId', concat('agent_', id),
    'modelName', 'HP_LASER',
    'serialNumber', concat('SN_', id),
    'familyName', 'LASER_FAMILY',
    'friendlyName', concat('Printer_', id),
    'dispErrorCode', 0
  ) AS deviceGeneral,

  named_struct(
    'connectionSetting', named_struct(
      'ipAddress', named_struct(
        'value', concat('192.168.1.', id % 255),
        'ipToLong', cast(id * 1000 as bigint)
      ),
      'port', named_struct(
        'http', 80,
        'https', 443
      ),
      'snmpSetting', named_struct(
        'snmpVersion', 'v2c',
        'retry', 3,
        'timeout', 5000,
        'getCommunityString', 'public',
        'contextName', 'default',
        'userName', 'snmpUser',
        'securityAuthType', 'SHA',
        'securityAuthKey', 'authKey',
        'securityPrivType', 'AES',
        'securityPrivKey', 'privKey'
      )
    ),
    'macAddress', concat('AA:BB:CC:DD:EE:', lpad(id, 2, '0')),
    'statusCode', 0,
    'status', array('READY'),
    'troubleCode', 0,
    'dskFlag', false,
    'location', 'OFFICE',
    'firmwareVersion', 'v1.0.0',
    'subnetMask', '255.255.255.0',
    'gateway', '192.168.1.1',
    'lastStatusUpdate', unix_timestamp() * 1000,
    'lastSuppliesUpdate', unix_timestamp() * 1000,
    'lastCounterUpdate', unix_timestamp() * 1000,
    'lastBasicUpdate', unix_timestamp() * 1000,
    'lifeCount', cast(id * 10000 as bigint),
    'detectedDeviceType', 'PRINTER',
    'productFamilyName', 'LASER_PRO',

    'forecastingList', array(
      named_struct(
        'color', 'BLACK',
        'printCount', id * 100,
        'tonerEndDate', current_timestamp(),
        'tonerNumber', 1,
        'tonerRemaining', 50
      )
    ),

    'interface', named_struct(
      'ethernetList', array(
        named_struct('id', 'eth0', 'type', 'ETHERNET', 'address', '00:11:22')
      ),
      'ipList', array(
        named_struct(
          'address', concat('192.168.1.', id % 255),
          'ethernetId', 'eth0',
          'subnetMask', '255.255.255.0',
          'defaultRoute', '192.168.1.1'
        )
      )
    ),

    'maintenanceCodeList', array(
      named_struct('id', 'M1', 'value', 'OK')
    ),

    'serviceCodeList', array(
      named_struct('id', 'S1', 'value', 'ACTIVE')
    ),

    'inTrayList', array(),
    'outTrayList', array(),
    'disposalMarkerList', array(),
    'markerList', array()
  ) AS printerDevice

FROM range(200);

In [0]:
select * from deviceData